# IMPORT

In [ ]:
import pandas as pd
import numpy as np
import os
import Levenshtein as lev
from rapidfuzz import fuzz
import re
from itertools import permutations

# CONFIG

In [ ]:
verbose=2

# FUNCTIONS

In [ ]:
def pulisci_stringhe_fuzz(df_temp, colonna, soglia_fuzz):
    '''
    Applies fuzzy matching to strings in a specified DataFrame column by comparing their uppercase forms. 
    If the similarity (via Levenshtein distance using `fuzz.ratio`) exceeds a given threshold, 
    the less frequent string is replaced with the more frequent one.

    Parameters:
    df_temp : DataFrame
        The DataFrame containing the column with the strings to analyze.
    colonna : str
        The name of the column in which the string values are compared and modified.
    soglia_fuzz : int
        The similarity threshold above which two strings are considered similar 
        enough to be grouped.

    Returns:
    df_temp2 : DataFrame
        A copy of the original DataFrame, with an additional column '<colonna>_mod' containing 
        the cleaned and standardized string values.
    varianti_dict : dict
        A dictionary where each key is a standardized string (the final form), and the corresponding 
        value is a list of original string variants that have been grouped under that key.
    '''
    df_temp2=df_temp.copy() #poi usare df_temp2
    colonna_mod = f'{colonna}_mod'
    df_temp2[colonna_mod] = df_temp2[colonna].str.upper()


    prima= set(df_temp2[colonna_mod].unique())

    diz = df_temp2[colonna_mod].value_counts().to_dict()


    for val1 in diz:
        for val2 in diz:
            if val1 != val2:
                similarità = fuzz.ratio(val1, val2)
                if similarità >= soglia_fuzz and diz[val1] >= diz[val2]:
                    df_temp2.loc[df_temp2[colonna_mod] == val2, colonna_mod] = val1  #Sostituisci tutti i valori uguali a val2 nella colonna colonna_mod del DataFrame df_temp2 con val1.# creare nuova colonna invece di replace
                elif similarità >=soglia_fuzz and diz[val1]<diz[val2]:
                    df_temp2.loc[df_temp2[colonna_mod] == val1, colonna_mod] = val2

   
    dopo= set(df_temp2[colonna_mod].unique())
    mancanti = prima - dopo
    #print("Valori rimossi o modificati:", mancanti, len(mancanti))

    varianti_dict = {
    key: sorted(set(s.upper() for s in group[colonna].unique() if isinstance(s, str)))
    for key, group in df_temp2.groupby(colonna_mod)}

     

    return df_temp2, varianti_dict 

In [ ]:
def pulisci_stringhe_fuzz2(df_temp, colonna, soglia_fuzz,colonna_da_uguagliare):
    '''
    Applies fuzzy matching to strings in a specified DataFrame column by comparing their uppercase forms. 
    If the similarity (via Levenshtein distance using `fuzz.ratio`) exceeds a given threshold and the strings in colonna_da_uguagliare are the same
    the less frequent string is replaced with the more frequent one.

    Parameters:
    df_temp : DataFrame
        The DataFrame containing the column with the strings to analyze.
    colonna : str
        The name of the column in which the string values are compared and modified.
    soglia_fuzz : int
        The similarity threshold above which two strings are considered similar 
        enough to be grouped.
    colonna_da_uguagliare: str
        The name of the column in which the strings have to be the same in order to modify one of them--> it's going to be the column of the state

    Returns:
    df_temp2 : DataFrame
        A copy of the original DataFrame, with an additional column '<colonna>_mod' containing 
        the cleaned and standardized string values.
    varianti_dict : dict
        A dictionary where each key is a standardized string (the final form), and the corresponding 
        value is a list of original string variants that have been grouped under that key.
    
    '''
    
    df_temp2=df_temp.copy() #poi usare df_temp2
    colonna_mod = f'{colonna}_mod'
    df_temp2[colonna_mod] = df_temp2[colonna].str.upper()


    prima= set(df_temp2[colonna_mod].unique())
    diz = df_temp2[colonna_mod].value_counts().to_dict()


    for val1 in diz:
        for val2 in diz:
            if val1 != val2:
                similarità = fuzz.ratio(val1, val2)
                if similarità >= soglia_fuzz and diz[val1] >= diz[val2]:
                    stato_val1 = df_temp2.loc[df_temp2[colonna_mod] == val1, colonna_da_uguagliare].iloc[0]
                    stato_val2 = df_temp2.loc[df_temp2[colonna_mod] == val2, colonna_da_uguagliare].iloc[0]
                    if stato_val1 == stato_val2:
                        df_temp2.loc[df_temp2[colonna_mod] == val2, colonna_mod] = val1  #Sostituisci tutti i valori uguali a val2 nella colonna colonna_mod del DataFrame df_temp2 con val1.# creare nuova colonna invece di replace
                elif similarità >=soglia_fuzz and diz[val1]<diz[val2]:
                    stato_val1 = df_temp2.loc[df_temp2[colonna_mod] == val1, colonna_da_uguagliare].iloc[0]
                    stato_val2 = df_temp2.loc[df_temp2[colonna_mod] == val2, colonna_da_uguagliare].iloc[0]
                    if stato_val1 == stato_val2:
                        df_temp2.loc[df_temp2[colonna_mod] == val1, colonna_mod] = val2

   
    dopo= set(df_temp2[colonna_mod].unique())
    mancanti = prima - dopo
    #print("Valori rimossi o modificati:", mancanti, len(mancanti))

    varianti_dict = {
    key: sorted(set(s.upper() for s in group[colonna].unique() if isinstance(s, str)))
    for key, group in df_temp2.groupby(colonna_mod)}
     

    return df_temp2, varianti_dict

In [ ]:
def pulisci_stringhe_lev(df_temp,colonna,soglia_lev):
    '''
    Applies Levenshtein distance to strings in a specified DataFrame column by comparing their uppercase forms. 
    If the distance exceeds a given threshold, the less frequent string is replaced with the more frequent one.

    Parameters:
    df_temp : DataFrame
        The DataFrame containing the column with the strings to analyze.
    colonna : str
        The name of the column in which the string values are compared and modified.
    soglia_lev : int
        The distance threshold above which two strings are considered similar 
        enough to be grouped.

    Returns:
    df_temp2 : DataFrame
        A copy of the original DataFrame, with an additional column '<colonna>_mod' containing 
        the cleaned and standardized string values.
    varianti_dict : dict
        A dictionary where each key is a standardized string (the final form), and the corresponding 
        value is a list of original string variants that have been grouped under that key.
    '''
    df_temp2=df_temp.copy()
    colonna_mod = f'{colonna}_mod'
    df_temp2[colonna_mod] = df_temp2[colonna].str.upper()
    

    prima= set(df_temp2[colonna_mod].unique())
    diz= df_temp2[colonna_mod].value_counts().to_dict()


    for val1 in diz:
        for val2 in diz:
            if val1!=val2:
                distanza=lev.distance(val1,val2)
                if distanza <= soglia_lev and diz[val1]>=diz[val2]:
                    df_temp2.loc[df_temp2[colonna_mod] == val2, colonna_mod] = val1
                elif distanza <= soglia_lev and diz[val1]<diz[val2]:
                    df_temp2.loc[df_temp2[colonna_mod] == val1, colonna_mod] = val2

    
    
    dopo= set(df_temp2[colonna_mod].unique())
    mancanti = prima - dopo
    #print("Valori rimossi o modificati:", mancanti, len(mancanti))

    varianti_dict = {
    key: sorted(set(s.upper() for s in group[colonna].unique() if isinstance(s, str)))
    for key, group in df_temp2.groupby(colonna_mod)}


    return df_temp2, varianti_dict

In [ ]:
def pulisci_stringhe_lev2(df_temp,colonna,soglia_lev,colonna_da_uguagliare):
    '''
    Applies Levenshtein distance to strings in a specified DataFrame column by comparing their uppercase forms. 
    If the distance exceeds a given threshold and the strings in colonna_da_uguagliare are the same, the less frequent string is replaced with the more frequent one.

    Parameters:
    df_temp : DataFrame
        The DataFrame containing the column with the strings to analyze.
    colonna : str
        The name of the column in which the string values are compared and modified.
    soglia_lev : int
        The distance threshold above which two strings are considered similar 
        enough to be grouped.
    colonna_da_uguagliare: The name of the column in which the strings have to be the same in order to modify one of them--> it's going to be the column of the state

    Returns:
    df_temp2 : DataFrame
        A copy of the original DataFrame, with an additional column '<colonna>_mod' containing 
        the cleaned and standardized string values.
    varianti_dict : dict
        A dictionary where each key is a standardized string (the final form), and the corresponding 
        value is a list of original string variants that have been grouped under that key.
    '''
    df_temp2=df_temp.copy()
    colonna_mod = f'{colonna}_mod'
    df_temp2[colonna_mod] = df_temp2[colonna].str.upper()
    

    prima= set(df_temp2[colonna_mod].unique())
    diz= df_temp2[colonna_mod].value_counts().to_dict()


    for val1 in diz:
        for val2 in diz:
            if val1!=val2:
                distanza=lev.distance(val1,val2)
                if distanza <= soglia_lev and diz[val1]>=diz[val2]:
                    stato_val1 = df_temp2.loc[df_temp2[colonna_mod] == val1, colonna_da_uguagliare].iloc[0]
                    stato_val2 = df_temp2.loc[df_temp2[colonna_mod] == val2, colonna_da_uguagliare].iloc[0]
                    if stato_val1 == stato_val2:
                        df_temp2.loc[df_temp2[colonna_mod] == val2, colonna_mod] = val1 
                elif distanza <= soglia_lev and diz[val1]<diz[val2]:
                    stato_val1 = df_temp2.loc[df_temp2[colonna_mod] == val1, colonna_da_uguagliare].iloc[0]
                    stato_val2 = df_temp2.loc[df_temp2[colonna_mod] == val2, colonna_da_uguagliare].iloc[0]
                    if stato_val1 == stato_val2:
                        df_temp2.loc[df_temp2[colonna_mod] == val1, colonna_mod] = val2

    
    
    dopo= set(df_temp2[colonna_mod].unique())
    mancanti = prima - dopo
    #print("Valori rimossi o modificati:", mancanti, len(mancanti))

    varianti_dict = {
    key: sorted(set(s.upper() for s in group[colonna].unique() if isinstance(s, str)))
    for key, group in df_temp2.groupby(colonna_mod)}

    return df_temp2, varianti_dict

In [ ]:
def pulisci_trattino(val):
    '''
    Replaces hyphens ('-') that are not at the beginning of the string with a dot ('.').

    Parameters:
    val: The input value, which will be converted to a string if not already.

    Returns:
    val: a string where all hyphens not at the start of the string have been replaced with dots.
    
    Notes:
    The regular expression `(?<!^)-` is a negative lookbehind assertion that ensures
    the hyphen is not at the start of the string before performing the substitution.
    '''
    val = str(val)
    val = re.sub(r'(?<!^)-', '.', val)
    return val

In [ ]:
def creo_dict(df,colonna_originale,colonna_modificata):
    '''
Creates a dictionary that maps each unique value in the modified column 
to a list of unique values from the original column that correspond to it.

Parameters:
- df: The input DataFrame containing the data.
- colonna_originale: The name of the column with the original, unmodified values.
- colonna_modificata: The name of the column with the processed or normalized values.

Returns:
- A dictionary where each key is a unique value from the modified column, and each value 
    is a list of unique original values grouped under that key.
    '''

    dizionario={key: list(group[colonna_originale].unique())
    for key, group in df.groupby(colonna_modificata)}
    return dizionario

In [ ]:
def creo_dict2(df, colonna_originale, colonna_modificata):
    '''
Generates a dictionary that maps each unique value in the modified column, 
rounded to two decimal places, to a list of unique corresponding values from the original column. 
It ensures numerical values are handled properly and preserves non-numeric values as they are.

Parameters:
- df: The input DataFrame containing the data.
- colonna_originale: The name of the column with the original values.
- colonna_modificata: The name of the column with the modified values to group by.

Returns:
- A dictionary where each key is a rounded (to 2 decimals) unique value from the modified column, 
  and the value is a list of unique corresponding entries from the original column.
'''

    dizionario = {
        round(key, 2): list(set(
            v if isinstance(v, (int, float)) else v
            for v in group[colonna_originale]
        ))
        for key, group in df.groupby(colonna_modificata)
    }
    return dizionario

In [ ]:
def uniforma_date_order_date(df, colonna_originale1, colonna_originale2):
    '''
Standardizes and corrects date values in a specified column by attempting to convert 
them into a consistent datetime format (YYYY/MM/DD). It creates two new columns: one in datetime 
format and one with the final corrected values.

For rows where the initial conversion fails (resulting in NaT), the function generates all possible 
permutations of the date components (e.g., day, month, year), tries to parse each permutation as 
a valid date, and selects the closest valid date that precedes or matches the date in a reference 
column (typically a shipment date).

Parameters:
- df: The input DataFrame.
- colonna_originale1: Name of the column containing potentially messy date values (e.g., order dates).
- colonna_originale2: Name of the reference date column to ensure logical ordering (e.g., ship dates).

Returns:
- df: The modified DataFrame, with two additional columns:
    - '<colonna_originale1>_dt': the parsed datetime column from colonna_originale1.
    - '<colonna_originale1>_def': the final corrected and standardized datetime values

    '''
    colonna1_dt=colonna_originale1 + '_dt'
    colonna2_dt=colonna_originale2 + '_dt'
    colonna1_def=colonna_originale1 + '_def'


    df[colonna1_dt] = pd.to_datetime(df[colonna_originale1], format="%d/%m/%Y",errors='coerce')
    df[colonna2_dt] = pd.to_datetime(df[colonna_originale2], format="%d/%m/%Y",errors='coerce')
    
    

    #Inizializzo colonna finale
    df[colonna1_def] = df[colonna1_dt]  # dove non è NaT, la data è già buona

    
    for i, row in df[pd.isna(df[colonna1_dt])].iterrows():
        raw_date = str(row[colonna_originale1]) #rendo la data in stringa così posso lavorarci
        display(f'La data iniziale è {raw_date}')
        display(f'La shipping date è {row[colonna2_dt]}')
        try:
            parts = re.split(r'\D+', raw_date.strip())
            display(f'Le parti sono {parts}')
             #cosi faccio lo split non solo sullo / ma qualsiasi simbolo non numerico
            combinazioni = ['/'.join(p) for p in permutations(parts)] #trovo tutte le combinazioni possibili
            display(f'Le combinazioni sono {combinazioni}')
            date_possibili = pd.to_datetime(combinazioni, errors='coerce') #rendo ogni combinazione una data in formato dt
            display(f'Le date possibili sono {date_possibili}')
            date_valide = date_possibili[date_possibili.notna() & (date_possibili <= row[colonna2_dt])] #controllo che la data possibile sia precedente alla data di spedizione
            display(f'Le date valide sono {date_valide}')
            if not date_valide.empty:
                distanze = row[colonna2_dt] - date_valide  # sarà sempre >= 0
                display(distanze)
                indice_minimo = distanze.argmin()
                display(indice_minimo)
                data_probabile = date_valide.iloc[indice_minimo] #indice della differenza minima
                display(f'La data corretta è {data_probabile}')
                df.at[i, colonna1_def] = data_probabile
            
            

    

        except:
            print(f'La riga {i} da errore')
    
    
        
    return df 

In [ ]:
def uniforma_date_ship_date(df:pd.DataFrame, colonna_originale1:str, colonna_originale2:str)->pd.DataFrame:
    '''
Attempts to clean and standardize date values in a specified column (e.g., shipping dates),
by converting them into a consistent datetime format (YYYY/MM/DD). It creates three new columns: one with 
the parsed datetime values from the original column, one for the reference string date column, and one containing 
the final corrected values.

When the conversion of a date fails (resulting in a NaT), the function generates all possible permutations 
of the numeric parts of the raw date string. It then filters these permutations to keep only valid dates 
that occur after or on the date specified in a reference column (typically an order date). Among these, 
the function selects the one closest in time to the reference date.

Parameters:
- df: The input DataFrame.
- colonna_originale1: The name of the column containing unclean or non-standard date strings (e.g., ship date).
- colonna_originale2: The name of the reference column used to validate dates chronologically (e.g., order date).

Returns:
- df: The modified DataFrame, enriched with the following columns:
    - '<colonna_originale1>_dt': parsed datetime version of colonna_originale1.
    - '<colonna_originale2>_dt': parsed datetime version of colonna_originale2.
    - '<colonna_originale1>_def': the cleaned and standardized date values for colonna_originale1.
'''

    #creo una colonna in formato datetime e una in formato stringa
    #quella in formato datetime mi serve per pulire il grosso degli errori
    #quella in formato stringa mi serve per pulire gli errori come per esempio se il mese supera il 12, o se è 0 ecc
    #quindi quando la colonna dt mi da errore, lavoro sulla colonna str
    # scelgo come formato unico il formato YYYY/MM/DD
    colonna1_dt=colonna_originale1 + '_dt'
    colonna2_dt=colonna_originale2 + '_dt'
    colonna1_def=colonna_originale1 + '_def'


    df[colonna1_dt] = pd.to_datetime(df[colonna_originale1], errors='coerce')
    df[colonna2_dt] = pd.to_datetime(df[colonna_originale2], errors='coerce')
    

    #Inizializzo colonna finale
    df[colonna1_def] = df[colonna1_dt]  # dove non è NaT, la data è già buona

    
    for i, row in df[pd.isna(df[colonna1_dt])].iterrows():
        raw_date = str(row[colonna_originale1]) #rendo la data in stringa così posso lavorarci
        try:
            parts = re.split(r'\D+', raw_date.strip()) #cosi faccio lo split non solo sullo / ma qualsiasi simbolo non numerico
            combinazioni = ['/'.join(p) for p in permutations(parts)] #trovo tutte le combinazioni possibili
            date_possibili = pd.to_datetime(combinazioni, errors='coerce') #rendo ogni combinazione una data in formato dt
            date_valide = date_possibili[date_possibili.notna() & (date_possibili >= row[colonna2_dt])] #controllo che la data possibile sia successiva alla data di ordine
            if not date_valide.empty:
                distanze = date_valide - row[colonna2_dt]  # sarà sempre >= 0
                indice_minimo = distanze.argmin()
                data_probabile = date_valide.iloc[indice_minimo] #indice della differenza minima
                df.at[i, colonna1_def] = data_probabile
        except:
            print(f'La riga {i} da errore')
    return df

In [ ]:
def verbose2(dizionario):
    '''
    Prints the values from a dictionary that are different from their corresponding key.

    Parameters:
    dizionario (dict): A dictionary where keys represent the cleaned or standardized values,
                       and values are lists of original values mapped to those keys.

    Returns:
    None: This function only prints output and does not return anything.
    '''
    for key, value_list in dizionario.items():
        # Stampiamo solo se la lista ha più di un elemento
        if len(value_list) > 1:
            # Troviamo i valori diversi dalla chiave
            diffs = [v for v in value_list if v != key]
            if diffs:  # Se ce ne sono almeno uno
                if len(diffs)==1:
                    print(f"Original value {diffs} is changed into {key}")
                else:
                    print(f"Original values {diffs} are changed into {key}")


# GET DATA

In [ ]:

# Leggi tutti i fogli come dizionario di DataFrame
xls = pd.ExcelFile("../data/raw/Superstore_mod_v1.xlsx")
df_dict = pd.read_excel(xls, sheet_name=None)  # None legge tutti i fogli

# df_dict è un dizionario con chiavi = nomi dei fogli, valori = DataFrame
for sheet_name, df in df_dict.items(): 
    globals()[f'df_{sheet_name.lower()}'] = df  # globals() in Python serve a restituire un dizionario contenente tutte le variabili globali 
                                                #attualmente disponibili nel contesto in cui viene chiamato
if verbose==2:
    print('Dataframe: countries')
    display(df_countries)
    print('Dataframe: orders')
    display(df_orders)
    print('Dataframe: customers')
    display(df_customers)
    print('Dataframe: products')
    display(df_products)

# CLEANING DATA
- for every column of every dataframe, a dictionary is created with each unique cleaned value and its original variants. This dictionary is converted into a dataframe too.

## COUNTRIES


In [ ]:
'''COLONNA STATE'''
if verbose>=1:
    print('When using the function `pulisci_stringhe_fuzz` with thresholds 90 and 80: NY and NC are not converted into NEW YORK and NORTH CAROLINA, so I corrected them manually.')
    print('When using the function `pulisci_stringhe_lev` with threshold = 1: NC was incorrectly matched to NEW YORK, so I used a manual mapping to fix it to NORTH CAROLINA')
    print("I choose the lev_1 one, as it is the one which returns the fewest errors.")
    print('') #per distaccare i print se verbose=1 dai print se verbose=2

df_countries_raw=df_countries.drop_duplicates() #tolgo i duplicati da tutto il df_countries

#FUNZIONE FUZZ SOGLIA 90
df_countries_fuzz_90, dict_countries_fuzz_90= pulisci_stringhe_fuzz(df_countries_raw,'State',90) 
df_countries_fuzz_90 = df_countries_fuzz_90.rename(columns={'State_mod': 'State_fuzz_90'})
df_countries_fuzz_90['State_fuzz_90'] = df_countries_fuzz_90['State_fuzz_90'].replace({
    'NY': 'NEW YORK', 
    'NC': 'NORTH CAROLINA'
})
df_stati_corretti_fuzz_90=pd.DataFrame([(k, v) for k, v in dict_countries_fuzz_90.items() if len(v)>1], columns=['State', 'Variants_fuzz_90'])


#FUNZIONE FUZZ SOGLIA 80
df_countries_fuzz_80,dict_countries_fuzz_80= pulisci_stringhe_fuzz(df_countries_fuzz_90,'State',80)
df_countries_fuzz_80= df_countries_fuzz_80.rename(columns={'State_mod':'State_fuzz_80'})
df_countries_fuzz_80['State_fuzz_80']=df_countries_fuzz_80['State_fuzz_80'].replace({'NY': 'NEW YORK', 'NC': 'NORTH CAROLINA'}) 
df_stati_corretti_fuzz_80=pd.DataFrame([(k, v) for k, v in dict_countries_fuzz_80.items() if len(v)>1], columns=['State', 'Variants_fuzz_80'])



#FUNZIONE LEV SOGLIA 1
df_countries_lev_1, dict_countries_lev_1=pulisci_stringhe_lev(df_countries_fuzz_80,'State',1) #applico la funzione lev
df_countries_lev_1= df_countries_lev_1.rename(columns={'State_mod':'State_lev_1'})
df_countries_lev_1['State_lev_1']=df_countries_lev_1['State_lev_1'].replace({'NY': 'NEW YORK'}) 
#df_countries_lev_1, guardando gli output, è quello che si avvicina di più ad essere corretto
state_map = {
    'NC': 'NORTH CAROLINA'
}
df_countries_lev_1['State_lev_1'] = df_countries_lev_1['State'].map(state_map).fillna(df_countries_lev_1['State_lev_1'])
df_stati_corretti_lev_1=pd.DataFrame([(k, v) for k, v in dict_countries_lev_1.items() if len(v) >1], columns=['State', 'Variants_lev_1'])

#MERGE TRA I 3 DF
df_merge_state_lev_fuzz90 = df_stati_corretti_fuzz_90.merge(df_stati_corretti_lev_1, on = 'State', how = 'outer')
df_merge_state_lev_fuzz90_fuzz80= df_merge_state_lev_fuzz90.merge(df_stati_corretti_fuzz_80, on='State', how='outer' )


if verbose ==2:
    print("Using fuzz function with threshold=90:")
    verbose2(dict_countries_fuzz_90)
    print('Using replace, i change NY in NEW YORK and NC in NORTH CAROLINA')
    print('')

    print("Using fuzz function with threshold=80:")
    verbose2(dict_countries_fuzz_80)
    print('Using replace, i change NY in NEW YORK and NC in NORTH CAROLINA')
    print('')

    print("Using levenshtein function with threshold=1:")
    verbose2(dict_countries_lev_1)
    print('Since NC is changed into NY, I use a map to fix it, and then i use .replace in order to change it in NEW YORK')
    print('')

    print("The 'cleaning' dataframe is:")
    display(df_merge_state_lev_fuzz90_fuzz80)
    


In [ ]:
'''COLONNA CITY'''
if verbose >=1:
    print("Using 'pulisci_stringhe_fuzz' with a threshold of 95, the cities 'BOWLINGGREEN', 'WOODSTOK', and 'ENDERSON' are not automatically corrected to 'BOWLING GREEN', 'WOODSTOCK', and 'HENDERSON'.")
    print("These cases are manually corrected")
    print("Among all approaches, the function with fuzziness threshold = 95 produces the best results")
    print("as the others cause too many incorrect city matches.")
    print('')

#FUNZIONE FUZZ CON SOGLIA 90
df_countries_fuzz_city_90, dict_countries_fuzz_city_90= pulisci_stringhe_fuzz(df_countries_lev_1,'City',90) 
df_countries_fuzz_city_90 = df_countries_fuzz_city_90.rename(columns={'City_mod': 'City_fuzz_90'})
df_countries_city_corrette_fuzz_90=pd.DataFrame([(k, v) for k, v in dict_countries_fuzz_city_90.items() if len(v)>1 ], columns=['City', 'Variants_fuzz_90'])


#FUNZIONE FUZZ CON SOGLIA 95
df_countries_fuzz_city_95,dict_countries_fuzz_city_95= pulisci_stringhe_fuzz(df_countries_fuzz_city_90,'City',95) 
df_countries_fuzz_city_95 = df_countries_fuzz_city_95.rename(columns={'City_mod': 'City_fuzz_95'})
df_countries_fuzz_city_95['City_fuzz_95']= df_countries_fuzz_city_95['City_fuzz_95'].replace({'BOWLINGGREEN': 'BOWLING GREEN'}).replace({'WOODSTOK': 'WOODSTOCK'})
df_countries_city_corrette_fuzz_95=pd.DataFrame([(k, v) for k, v in dict_countries_fuzz_city_95.items() if len(v)>1 ], columns=['City', 'Variants_fuzz_95'])
df_countries_fuzz_city_95 = df_countries_fuzz_city_95.rename(columns={'City_fuzz_95': 'City_def'})
df_countries_fuzz_city_95['City_def']=df_countries_fuzz_city_95['City_def'].replace({
    'ENDERSON':'HENDERSON'
})



#FUNZIONE LEV CON SOGLIA 1
df_countries_lev_city_1, dict_countries_lev_city_1= pulisci_stringhe_lev(df_countries_fuzz_city_95,'City',1) 
df_countries_lev_city_1 = df_countries_lev_city_1.rename(columns={'City_mod': 'City_lev_1'})
df_countries_city_corrette_lev_1=pd.DataFrame([(k, v) for k, v in dict_countries_lev_city_1.items() if len(v)>1], columns=['City', 'Variants_lev_1'])


#MERGE TRA I 3 DF
df_merge_city_fuzz_95_fuzz_90=df_countries_city_corrette_fuzz_95.merge(df_countries_city_corrette_fuzz_90, on='City', how='outer')
df_merge_city_fuzz_95_fuzz_90_lev_1=df_countries_city_corrette_lev_1.merge(df_merge_city_fuzz_95_fuzz_90, on='City', how='outer')


#METODO PIù GIUSTO: FUZZ CON SOGLIA=95 ci sono pochi errori in questa colonna city, ce ne sono molti nella colonna city del foglio orders, ancora da finire
df_countries_lev_city_1=df_countries_lev_city_1[['Country','City','City_fuzz_90','City_def','City_lev_1','State','State_fuzz_90','State_fuzz_80','State_lev_1','Postal Code','Region']]

if verbose==2:
    print("Using fuzz function with threshold=90: ")
    verbose2(dict_countries_fuzz_city_90)
    print('')

    print("Using fuzz function with threshold=95:")
    verbose2(dict_countries_fuzz_city_95)
    print('I choose this function because is the one with least errors')
    print('')

    print("Using levenshtein function with threshold=1:")
    verbose2(dict_countries_lev_city_1)
    print('')

    print("The 'cleaning' dataframe is:")
    display(df_merge_city_fuzz_95_fuzz_90_lev_1)





Using 'pulisci_stringhe_fuzz' with a threshold of 95, the cities 'BOWLINGGREEN', 'WOODSTOK', and 'ENDERSON' are not automatically corrected to 'BOWLING GREEN', 'WOODSTOCK', and 'HENDERSON'.
These cases are manually corrected
Among all approaches, the function with fuzziness threshold = 95 produces the best results
as the others cause too many incorrect city matches.

Using fuzz function with threshold=90: 
Original value ['BOWLING GREEN'] is changed into BOWLINGGREEN
Original value ['AMARILLO'] is changed into CAMARILLO
Original value ['HENDERSON'] is changed into ENDERSON
Original value ['OVERLAND PARK'] is changed into ORLAND PARK
Original value ['EDMOND'] is changed into REDMOND
Original value ['TRENTON'] is changed into RENTON
Original value ['WOODSTOCK'] is changed into WOODSTOK

Using fuzz function with threshold=95:
Original value ['BOWLING GREEN'] is changed into BOWLINGGREEN
I choose this function because is the one with least errors

Using levenshtein function with threshold=

,City,Variants_lev_1,Variants_fuzz_95,Variants_fuzz_90
0,BEDFORD,"[BEDFORD, MEDFORD]",NaN,NaN
1,BOWLINGGREEN,"[BOWLING GREEN, BOWLINGGREEN]","[BOWLING GREEN, BOWLINGGREEN]","[BOWLING GREEN, BOWLINGGREEN]"
2,CAMARILLO,"[AMARILLO, CAMARILLO]",NaN,"[AMARILLO, CAMARILLO]"
3,CLIFTON,"[CLIFTON, CLINTON]",NaN,NaN
4,ENDERSON,"[ENDERSON, HENDERSON]",NaN,"[ENDERSON, HENDERSON]"
5,ENGLEWOOD,"[ENGLEWOOD, INGLEWOOD]",NaN,NaN
6,LAYTON,"[LAWTON, LAYTON]",NaN,NaN
7,MACON,"[MACON, MASON]",NaN,NaN
8,MONROE,"[CONROE, MONROE]",NaN,NaN
9,NORMAL,"[NORMAL, NORMAN]",NaN,NaN


In [ ]:
''' ANCORA COLONNA CITY MA CON CONTROLLO DELLO STATO PER MODIFICARE LA CITTA: MODIFICO LA CITTA' SOLO SE HANNO LO STATO UGUALE'''

if verbose>=1:
    print("In this section, I apply string-cleaning functions that modify city names only if the corresponding state (column 'State_lev_1') is the same.")
    print("This helps avoid incorrect matches between cities with the same or similar names but located in different states.")
    print("I use the modified versions of the functions (pulisci_stringhe_fuzz2 and pulisci_stringhe_lev2), which include an additional state-level condition.")

#METODO FUZZ (2) CON SOGLIA 90
df_countries_fuzz_city_90_c, dict_countries_fuzz_city_90_c= pulisci_stringhe_fuzz2(df_countries_lev_1,'City',90,'State_lev_1') 
df_countries_fuzz_city_90_c = df_countries_fuzz_city_90_c.rename(columns={'City_mod': 'City_fuzz_90'})
df_countries_city_corrette_fuzz_90_c=pd.DataFrame([(k, v) for k, v in dict_countries_fuzz_city_90_c.items() if len(v)>1], columns=['City', 'Variants_fuzz_90'])


#METODO FUZZ (2) CON SOGLIA 95
df_countries_fuzz_city_95_c,dict_countries_fuzz_city_95_c= pulisci_stringhe_fuzz2(df_countries_fuzz_city_90_c,'City',95,'State_lev_1') 
df_countries_fuzz_city_95_c = df_countries_fuzz_city_95_c.rename(columns={'City_mod': 'City_fuzz_95'})
df_countries_city_corrette_fuzz_95_c=pd.DataFrame([(k, v) for k, v in dict_countries_fuzz_city_95_c.items() if len(v)>1], columns=['City', 'Variants_fuzz_95'])



#METODO LEV (2) CON SOGLIA 1
df_countries_lev_city_1_c, dict_countries_lev_city_1_c= pulisci_stringhe_lev2(df_countries_fuzz_city_95_c,'City',1,'State_lev_1') 
df_countries_lev_city_1_c = df_countries_lev_city_1_c.rename(columns={'City_mod': 'City_lev_1'})
df_countries_city_corrette_lev_1_c=pd.DataFrame([(k, v) for k, v in dict_countries_lev_city_1_c.items() if len(v)>1], columns=['City', 'Variants_lev_1'])


#MERGE TRA I 3 DF
df_merge_city_fuzz_95_fuzz_90_c=df_countries_city_corrette_fuzz_95_c.merge(df_countries_city_corrette_fuzz_90_c, on='City', how='outer')
df_merge_city_fuzz_95_fuzz_90_lev_1_c=df_countries_city_corrette_lev_1_c.merge(df_merge_city_fuzz_95_fuzz_90_c, on='City', how='outer')

if verbose==2:
    print("Using fuzz function with threshold=90, checking the state as well, the 'cleaning' dataframe is:")
    verbose2(dict_countries_fuzz_city_90_c)
    print('')

    print("Using fuzz function with threshold=95, checking the state as well, the 'cleaning' dataframe is:")
    verbose2(dict_countries_fuzz_city_95_c)
    print('')

    print("Using levenshtein function with threshold=1, checking the state as well, the 'cleaning' dataframe is:")
    verbose2(dict_countries_lev_city_1_c)
    print('')

    print("The 'cleaning' dataframe is:")
    display(df_merge_city_fuzz_95_fuzz_90_lev_1_c)

## ORDERS



In [ ]:
df_orders_raw=df_orders
df_orders_mod=df_orders_raw.copy().drop_duplicates()

'''COLONNA SHIP MODE'''  
#METODO LEV CON SOGLIA 5
df_orders_ship_mode_lev_1, dict_orders_ship_mode_lev_1 =pulisci_stringhe_lev(df_orders_mod,'Ship Mode',5)
df_ship_mode_mod_corrette=pd.DataFrame([(k, v) for k, v in dict_orders_ship_mode_lev_1.items() if len(v)>1], columns=['Ship_mode', 'Variants'])

mask_2_class = df_orders_ship_mode_lev_1['Ship Mode'] == '2 Class'
df_orders_ship_mode_lev_1.loc[mask_2_class, 'Ship Mode_mod'] = 'SECOND CLASS'

#display(df_orders_ship_mode_lev_1.loc[df_orders_ship_mode_lev_1['Ship Mode']=='2 Class'])


if verbose==2:
    print("Using levenshtein function with threshold=5, the 'cleaning' dataframe is:")
    display(df_ship_mode_mod_corrette)
    verbose2(dict_orders_ship_mode_lev_1)
    print("Since it mistakenly change '2 CLASS' in 'FIRST CLASS', I manually fix it in the dataframe")


In [ ]:
'''COLONNA SALES'''
if verbose>=1:
    print(" The `pulisci_trattino` function is applied to clean the 'Sales' column, replacing dashes (`-`) that are not at the beginning of the string.")
    print("Then, additional cleaning is done replacing commas (`,`) and apostrophes (`'`) with dots (`.`) and to remove any unintended characters like `a`.")

df_orders_ship_mode_lev_1['Sales_mod']= df_orders_ship_mode_lev_1['Sales'].apply(pulisci_trattino) # sostitiusco il - con il . quando è in mezzo alla stringa
df_orders_ship_mode_lev_1['Sales_mod'] = df_orders_ship_mode_lev_1['Sales_mod'].astype(str).str.strip() # rimuovo eventuali spazi bianchi e converto in string
df_orders_ship_mode_lev_1['Sales_mod'] = df_orders_ship_mode_lev_1['Sales_mod'].str.replace(",", '.', regex=False).str.replace("'", '.', regex=False).str.replace('a', '', regex=False) # sostituisco le virgole con punti (per uniformare la notazione decimale), il ' e la a

df_orders_ship_mode_lev_1['Sales_mod'] = pd.to_numeric(df_orders_ship_mode_lev_1['Sales_mod'], errors='coerce')
dict_sales_mod = creo_dict2(df_orders_ship_mode_lev_1,'Sales','Sales_mod')
df_sales_mod_corrette=pd.DataFrame([(k, v) for k, v in dict_sales_mod.items() if len(v)>1], columns=['Sales', 'Variants'])


df_orders_ship_mode_lev_1=df_orders_ship_mode_lev_1[['Order ID','Order Date','Ship Date','Ship Mode','Ship Mode_mod','Customer ID','City','Postal Code','Product ID', 'Sales','Sales_mod','Quantity','Discount','Profit']] #avvicino Sales_mod a Sales

if verbose==2:
    print("The 'cleaning' dataframe is:")
    display(df_sales_mod_corrette)
    verbose2(dict_sales_mod)


In [ ]:
'''COLONNA QUANTITY'''
if verbose>=1:
    print("The `pulisci_trattino` function is applied to clean the 'Quantity' column, replacing dashes (`-`) that are not at the beginning of the string.")
    print("Then, additional cleaning is done replacing commas (`,`) and apostrophes (`'`) with dots (`.`) and to remove any unintended characters like `ì`.")

    
df_orders_ship_mode_lev_1['Quantity_mod']=df_orders_ship_mode_lev_1['Quantity'].astype(str).apply(pulisci_trattino)
df_orders_ship_mode_lev_1['Quantity_mod'] = df_orders_ship_mode_lev_1['Quantity_mod'].str.replace(",", '.', regex=False).str.replace("'", '.', regex=False).str.replace('ì', '', regex=False) # sostituisco le virgole con punti (per uniformare la notazione decimale), l'apostrofo ' e la ì

df_orders_ship_mode_lev_1['Quantity_mod'] = pd.to_numeric(df_orders_ship_mode_lev_1['Quantity_mod'], errors='coerce')
dict_quantity_mod=creo_dict2(df_orders_ship_mode_lev_1,'Quantity','Quantity_mod')
df_quantity_mod_corrette=pd.DataFrame([(k, v) for k, v in dict_quantity_mod.items() if len(v)>1], columns=['Quantity', 'Variants'])

if verbose==2:
    print('The cleaning dataframe is:')
    display(df_quantity_mod_corrette)
    verbose2(dict_quantity_mod)


In [ ]:
'''COLONNA DISCOUNT'''
if verbose>=1:
    print("Cleaning is done using '.replace()' to remove any unintended characters like 'ù' and '%'.")
df_orders_ship_mode_lev_1['Discount_mod'] = df_orders_ship_mode_lev_1['Discount'].astype(str).str.replace("ù", '', regex=False).str.replace("%", '', regex=False)

df_orders_ship_mode_lev_1['Discount_mod']=pd.to_numeric(df_orders_ship_mode_lev_1['Discount_mod'],errors='coerce')
dict_discount_mod=creo_dict(df_orders_ship_mode_lev_1,'Discount','Discount_mod')
df_discount_mod_corretto=pd.DataFrame([(k, v) for k, v in dict_discount_mod.items() if len(v)>1], columns=['Discount', 'Variants'])

if verbose==2:
    print("The 'cleaning' dataframe is:")
    display(df_discount_mod_corretto)
    verbose2(dict_discount_mod)


In [ ]:
'''COLONNA PROFIT'''
if verbose>=1:
    print("The `pulisci_trattino` function is applied to clean the 'Profit' column, replacing dashes (`-`) that are **not at the beginning** of the string.")
    print("Then, additional cleaning is done using `.replace()` to: replace commas (`,`) and apostrophes (`'`) with dots (`.`) and to remove any unintended characters like `a`.")

df_orders_ship_mode_lev_1['Profit_mod']= df_orders_ship_mode_lev_1['Profit'].astype(str).apply(pulisci_trattino)
df_orders_ship_mode_lev_1['Profit_mod'] = df_orders_ship_mode_lev_1['Profit_mod'].str.strip() # 1. Rimuovi eventuali spazi bianchi e converti in string
df_orders_ship_mode_lev_1['Profit_mod'] = df_orders_ship_mode_lev_1['Profit_mod'].str.replace(",", '.', regex=False).str.replace("'", '.', regex=False).str.replace('a', '', regex=False) # 2. Sostituisci le virgole con punti (per uniformare la notazione decimale)

df_orders_ship_mode_lev_1['Profit_mod'] = pd.to_numeric(df_orders_ship_mode_lev_1['Profit_mod'], errors='coerce')
dict_profit_mod=creo_dict2(df_orders_ship_mode_lev_1,'Profit','Profit_mod')
df_profit_mod_corretto=pd.DataFrame([(k, v) for k, v in dict_profit_mod.items() if len(v)>1 ], columns=['Profit', 'Variants'])

if verbose==2:
    print("The 'cleaning' dataframe is:")
    display(df_profit_mod_corretto)
    verbose2(dict_profit_mod)


In [ ]:
'''COLONNA CITY'''

if verbose >= 1:
    print("Cleaning the 'City' column using string similarity functions (fuzz and levenshtein) with different thresholds.")
    print("- First, I apply 'pulisci_stringhe_fuzz' with thresholds 90 and 85.")
    print("- Then, I use 'pulisci_stringhe_lev' with threshold=1, which gives the best results with the least errors.")
    print("- Despite that, some cities are still mistakenly changed or left unchanged:")
    print("  - I manually restore wrongly modified cities using a mask.")
    print("  - I manually correct specific city names not automatically fixed.")
    print("- Finally, I also test 'pulisci_stringhe_lev' with threshold=2, but it introduces too many false corrections.")


#FUNZIONE FUZZ CON SOGLIA 90
df_orders_fuzz_city_90, dict_orders_city_fuzz_90= pulisci_stringhe_fuzz(df_orders_ship_mode_lev_1,'City',90) 
df_orders_fuzz_city_90 = df_orders_fuzz_city_90.rename(columns={'City_mod': 'City_fuzz_90'})
df_orders_fuzz_city_90['City_fuzz_90']= df_orders_fuzz_city_90['City_fuzz_90'].replace({'BOWLINGGREEN': 'BOWLING GREEN'})
df_orders_city_corrette_fuzz_90=pd.DataFrame([(k, v) for k, v in dict_orders_city_fuzz_90.items() if len(v)>1], columns=['City', 'Variants_fuzz_90'])

#FUNZIONE FUZZ CON SOGLIA 85
df_orders_fuzz_city_85,dict_orders_city_fuzz_85=pulisci_stringhe_fuzz(df_orders_fuzz_city_90,'City',85) 
df_orders_fuzz_city_85 = df_orders_fuzz_city_85.rename(columns={'City_mod': 'City_fuzz_85'})
df_orders_city_corrette_fuzz_85=pd.DataFrame([(k, v) for k, v in dict_orders_city_fuzz_85.items() if len(v)>1], columns=['City', 'Variants_fuzz_85'])



#FUNZIONE LEV CON SOGLIA 1
df_orders_lev_city_1,dict_orders_city_lev_1=pulisci_stringhe_lev(df_orders_fuzz_city_85,'City',1) # quello con meno errori
df_orders_lev_city_1 = df_orders_lev_city_1.rename(columns={'City_mod': 'City_lev_1'})
citta_da_ripristinare = ['Conroe', 'Redding', 'Camarillo', 'Englewood', 'Mason', 'Medford', 'Normal', 'Layton', 'Renton','Clifton','Edmond'] #SISTEMO LE CITTà CHE HA RIMOSSO MA NON DOVEVA
mask = df_orders_lev_city_1['City'].isin(citta_da_ripristinare)
df_orders_lev_city_1.loc[mask, 'City_lev_1'] = df_orders_lev_city_1.loc[mask, 'City'].str.upper()
df_orders_city_corrette_lev_1=pd.DataFrame([(k, v) for k, v in dict_orders_city_lev_1.items() if len(v)>1], columns=['City', 'Variants_lev_1'])
df_orders_lev_city_1['City_lev_1']=df_orders_lev_city_1['City_lev_1'].replace({
    'ALECSANDRIA':'ALEXANDRIA',
    'LA':'LOS ANGELES',
    'MARLBOROUG':'MARLBOROUGH',
    'FILADELPHIA':'PHILADELPHIA',
    'NY CITY':'NEW YORK CITY',
    'NYC':'NEW YORK CITY',
    'EDINBURG':'EDINBURGH'
})
df_orders_lev_city_1=df_orders_lev_city_1.rename(columns={'City_lev_1':'City_def'})


#FUNZIONE LEV CON SOGLIA 2
df_orders_lev_city_2,dict_orders_city_lev_2=pulisci_stringhe_lev(df_orders_lev_city_1,'City',2) #non va bene, rimuove troppe città corrette
df_orders_lev_city_2 = df_orders_lev_city_2.rename(columns={'City_mod': 'City_lev_2'})
df_orders_city_corrette_lev_2=pd.DataFrame([(k, v) for k, v in dict_orders_city_lev_2.items() if len(v)>1], columns=['City', 'Variants_lev_2'])



#FACCIO IL MERGE TRA I 4 DF
df_merge_orders_city_fuzz_90_fuzz_85=df_orders_city_corrette_fuzz_90.merge(df_orders_city_corrette_fuzz_85, on='City',how='outer')
df_merge_orders_city_fuzz_90_fuzz_85_lev_1=df_orders_city_corrette_lev_1.merge(df_merge_orders_city_fuzz_90_fuzz_85, on='City', how='outer')
df_merge_orders_city_fuzz_90_fuzz_85_lev_1_lev_2=df_orders_city_corrette_lev_2.merge(df_merge_orders_city_fuzz_90_fuzz_85_lev_1, on='City', how='outer')

if verbose==2:
    print("The 'cleaning' dataframe is:")
    display(df_merge_orders_city_fuzz_90_fuzz_85_lev_1_lev_2)

In [ ]:
'''COLONNA ORDER DATE'''
if verbose>=1:
    print("Cleaning the 'Order Date' column using the 'uniforma_date_order_date' function.")
    print("Some dates cannot be parsed correctly using standard permutations (e.g., YYYY-MM-DD).")
    print("For these specific cases (e.g., '13/13/2014'), I manually correct the values.")
    print("Then, I drop temporary columns ")


df_date_pulite=uniforma_date_order_date(df_orders_lev_city_1,'Order Date','Ship Date')
display(df_date_pulite)

dict_order_date=creo_dict(df_date_pulite,'Order Date','Order Date_def')

df_order_date_corrette=pd.DataFrame([(k, v) for k, v in dict_order_date.items() if len(v)>1], columns=['Order Date', 'Variants'])

#C'è ANCORA IL PROBLEMA DELLE RIGHE COME RIGA (422 OPPURE 513) CON LA DATA 13/13/2014 CHE ANCHE SE GUARDO LE DIVERSE COMBINAZIONI, NON HA NESSUNA DATA VALIDA, PER QUESTE DUE USO IL REPLACE
df_date_pulite.loc[df_date_pulite['Order Date'] == '13/13/2014', ['Order Date_dt', 'Order Date_def']] = pd.Timestamp('2014-10-13')
df_date_pulite.loc[df_date_pulite['Order Date'] == '25/13/2011', ['Order Date_dt', 'Order Date_def']] = pd.Timestamp('2011-11-25')
df_date_pulite.loc[df_date_pulite['Order Date'] == '14/03/013', ['Order Date_dt', 'Order Date_def']] = pd.Timestamp('2013-03-14')

df_date_pulite=df_date_pulite.drop(columns=['Order Date_dt','Ship Date_dt'])

#if verbose==2:
    #print("The 'cleaning' dataframe is:")
    #display(df_order_date_corrette)
    #verbose2(dict_order_date)



Cleaning the 'Order Date' column using the 'uniforma_date_order_date' function.
Some dates cannot be parsed correctly using standard permutations (e.g., YYYY-MM-DD).
For these specific cases (e.g., '13/13/2014'), I manually correct the values.
Then, I drop temporary columns 
The 'cleaning' dataframe is:


/var/folders/92/7kxqqyld1rl76prg4ll1c3s40000gn/T/ipykernel_87334/40077103.py:41: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  date_possibili = pd.to_datetime(combinazioni, errors='coerce') #rendo ogni combinazione una data in formato dt
/var/folders/92/7kxqqyld1rl76prg4ll1c3s40000gn/T/ipykernel_87334/40077103.py:41: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  date_possibili = pd.to_datetime(combinazioni, errors='coerce') #rendo ogni combinazione una data in formato dt
/var/folders/92/7kxqqyld1rl76prg4ll1c3s40000gn/T/ipykernel_87334/40077103.py:41: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, plea

,Order Date,Variants
0,2011-12-31,"[2011-12-31 00:00:00, 12/31/2011]"
1,2012-01-17,"[17/Jan/2012, 2012-01-17 00:00:00]"
2,2013-09-18,"[18/9/2013, 2013-09-18 00:00:00]"
3,2014-03-05,"[2014-03-05 00:00:00, 5th March 14]"
4,2014-04-07,"[2014-04-07 00:00:00, 04/07/14]"
5,2014-08-08,"[2014-08-08 00:00:00, 8/08/2014]"
6,2014-09-29,"[2014-09-29 00:00:00, 29-09-2014]"
7,2014-11-09,"[2014-11-09 00:00:00, 11/9/2014]"


Original value ['12/31/2011'] is changed into 2011-12-31 00:00:00
Original value ['17/Jan/2012'] is changed into 2012-01-17 00:00:00
Original value ['18/9/2013'] is changed into 2013-09-18 00:00:00
Original value ['5th March 14'] is changed into 2014-03-05 00:00:00
Original value ['04/07/14'] is changed into 2014-04-07 00:00:00
Original value ['8/08/2014'] is changed into 2014-08-08 00:00:00
Original value ['29-09-2014'] is changed into 2014-09-29 00:00:00
Original value ['11/9/2014'] is changed into 2014-11-09 00:00:00


In [ ]:
'''COLONNA SHIP DATE'''
if verbose>=1:
    print("Cleaning the 'Ship Date' column using the 'uniforma_date_ship_date' function.")
    print("Some dates cannot be parsed correctly using standard permutations (e.g., YYYY-MM-DD).")
    print("For these specific cases (e.g., '22/0/2013'), I manually correct the values.")
    print("Then, I drop temporary columns ")

df_date_pulite2=uniforma_date_ship_date(df_date_pulite,'Ship Date', 'Order Date')
dict_ship_date=creo_dict(df_date_pulite2,'Ship Date','Ship Date_def')
df_ship_date_corrette=pd.DataFrame([(k, v) for k, v in dict_ship_date.items() if len(v)>1], columns=['Ship Date', 'Variants'])


#devo pulire ancora le date in cui nessuna combniazione mi da una data possibile oppure altri errori
df_date_pulite2.loc[df_date_pulite2['Ship Date'] == '02 Sép 2013', ['Ship Date_dt', 'Ship Date_def']] = pd.Timestamp('2013-09-02')
df_date_pulite2.loc[df_date_pulite2['Ship Date'] == '22/0/2013', ['Ship Date_dt', 'Ship Date_def']] = pd.Timestamp('2013-10-22')
df_date_pulite2.loc[df_date_pulite2['Ship Date'] == '28/13/2013', ['Ship Date_dt', 'Ship Date_def']] = pd.Timestamp('2013-05-28')
df_date_pulite2.loc[df_date_pulite2['Ship Date'] == '15/09/013', ['Ship Date_dt', 'Ship Date_def']] = pd.Timestamp('2013-09-15')
df_date_pulite2.loc[df_date_pulite2['Ship Date'] == '10_12_2012', ['Ship Date_dt', 'Ship Date_def']] = pd.Timestamp('2012-12-10')
df_date_pulite2=df_date_pulite2.drop(columns=['Ship Date_dt','Order Date_dt'])

if verbose==2:
    print("The 'cleaning' dataframe is:")
    display(df_ship_date_corrette)
    verbose2(dict_ship_date)
    


In [ ]:
'''COLONNA PRODUCT ID'''
if verbose >= 1:
    print("Cleaning the 'Product ID' column by removing unwanted characters like 'ù'.")
    print("Afterwards, the string is reformatted into the standard structure 'AAA-BB-1234567'.")

df_date_pulite2['Product ID_def']=df_date_pulite2['Product ID'].copy()

df_date_pulite2['Product ID_def'] = (
    df_date_pulite2['Product ID_def']
    .str.replace('ù', '', regex=False)
    .str.upper()
    .str.replace(' ', '', regex=False)
    .str.replace('-', '', regex=False)
)


#Applico il formato AAA-BB-1234567
df_date_pulite2['Product ID_def'] = (
    df_date_pulite2['Product ID_def'].str[:3] + '-' +
    df_date_pulite2['Product ID_def'].str[3:5] + '-' +
    df_date_pulite2['Product ID_def'].str[5:]
)
if verbose==2:
    print('The dataframe with reformatted Product ID column is:')
    display(df_date_pulite2)
    

## PRODUCTS


In [ ]:
df_products_raw = df_products
df_products_mod=df_products_raw.copy().drop_duplicates()


'''COLONNA CATEGORY'''
#METODO LEV CON SOGLIA 6
df_products_cat_lev_6, dict_products_cat=pulisci_stringhe_lev(df_products_mod,'Category',6)
df_products_cat_corrette=pd.DataFrame([(k, v) for k, v in dict_products_cat.items()  if len(v)>1], columns=['Category', 'Variants'])

if verbose==2:
    print("Using levenshtein function with threshold=6, the 'cleaning' dataframe is: ")
    display(df_products_cat_corrette)
    verbose2(dict_products_cat)
    

In [ ]:
'''COLONNA SUB-CATEGORY'''
#METODO LEV CON SOGLIA 2
df_products_sub_lev2, dict_products_sub=pulisci_stringhe_lev(df_products_cat_lev_6,'Sub-Category',2)
df_products_sub_corrette=pd.DataFrame([(k, v) for k, v in dict_products_sub.items() if len(v)>1], columns=['Sub-Category', 'Variants'])


df_products_sub_lev2=df_products_sub_lev2[['Product ID','Category','Category_mod','Sub-Category','Sub-Category_mod','Product Name']]#cambio ordine delle colonnne

if verbose==2:
    print("Using levenshtein function with threshold=2, the 'cleaning' dataframe is: ")
    display(df_products_sub_corrette)
    verbose2(dict_products_sub)

In [ ]:
'''COLONNA PRODUCT-ID'''
if verbose >= 1:
    print("In this section, the structure of 'Product ID' is validated based on 'Category' and 'Sub-Category'.")
    print("Expected format: the ID should be composed as 'XXX-YY' where:")
    print("- 'XXX' matches the first 3 letters of 'Category'")
    print("- 'YY' matches the first 2 letters of 'Sub-Category'")
    

df_products_sub_lev2['Product ID_mod']=df_products_sub_lev2['Product ID']

for index, row in df_products_sub_lev2.iterrows():
    product_id = row['Product ID_mod']
    category = row['Category_mod']
    sub_category= row['Sub-Category_mod']

    if pd.notnull(product_id) and '-' in product_id:
        product_cat = product_id.split('-')[0]
        product_sub_cat = product_id.split('-')[1]

        if product_cat != category[:3]:  # primi 3 caratteri della stringa
            print(f'La riga {index} è sbagliata: ID = {product_id}, Categoria = {category}')
        
        elif product_sub_cat != sub_category[:2]:
            print(f'La riga {index} è sbagliata: ID= {product_id}, Sub-Category= {product_sub_cat}')
            #OK NO ERRORI


## CUSTOMERS


In [ ]:
df_customers_raw=df_customers
df_customers_mod=df_customers_raw.copy().drop_duplicates()

'''COLONNA SEGMENT'''
#METODO LEV CON SOGLIA 2
df_customers_lev_2, dict_segment=pulisci_stringhe_lev(df_customers_mod,'Segment',2)
df_customers_segment_corretto=pd.DataFrame([(k, v) for k, v in dict_segment.items() if len(v)>1], columns=['Segment', 'Variants'])

if verbose==2:
    print("Using levenshtein function with threshold=2, the 'cleaning' dataframe is: ")
    display(df_customers_segment_corretto)
    verbose2(dict_segment)


In [ ]:
if verbose >= 1:
    print("In this section, Customer IDs are validated and corrected based on Customer Names.")
    print("Steps performed:")
    print("- Hyphens (`-`) and underscores (`_`) in 'Customer Name' are replaced with spaces.")
    print("- Each 'Customer ID' is expected to have the first letter of the name and surname.")
    print("- If the initials in the ID do not match the name, the ID is corrected accordingly.")
    print("- Some compound names (e.g., 'CoreyLock') are split using capital letters to extract initials.")
    

df_customers_lev_2['Customer ID_mod']=df_customers_lev_2['Customer ID'].copy()
df_customers_lev_2['Customer Name_mod'] = (df_customers_lev_2['Customer Name'].str.replace('-', ' ', regex=False).str.replace('_', ' ', regex=False))

'''COLONNA CUSTOMER ID'''
for index in df_customers_lev_2.index:
    customer_id = df_customers_lev_2.at[index, 'Customer ID_mod']
    customer_full_name = df_customers_lev_2.at[index, 'Customer Name_mod']
    
    if pd.notnull(customer_id) and '-' in customer_id:
        customer_id_first_letter_name = customer_id[0]
        customer_id_first_letter_surname = customer_id[1]

        parts = customer_full_name.strip().split()
        if len(parts) >= 2:
            first_letter_name = parts[0][0]
            first_letter_surname = parts[1][0]
        else:
            # Gestione nome-cognome attaccati tipo 'CoreyLock'
            split_parts = re.split(r'(?=[A-Z])', customer_full_name)
            if len(split_parts) >= 3:
                first_letter_name = split_parts[1][0]
                first_letter_surname = split_parts[2][0]
            else:
                continue  # nome non gestibile, salta

        if (
            customer_id_first_letter_name != first_letter_name or
            customer_id_first_letter_surname != first_letter_surname
        ):
            print(f'La riga {index} è errata: il Customer ID è {customer_id}, il Customer Name è {customer_full_name}')
            df_customers_lev_2.at[index, 'Customer ID_mod'] = first_letter_name + first_letter_surname + customer_id[2:]

dict_customers_id_corretti=creo_dict(df_customers_lev_2,'Customer ID','Customer ID_mod')
df_customers_id_corretti=pd.DataFrame([(k, v) for k, v in dict_customers_id_corretti.items() if len(v)>1], columns=['Customer ID', 'Variants'])

df_customers_lev_2=df_customers_lev_2.rename(columns={'Customer ID_mod':'Customer ID_def'})
if verbose==2:
    print("The 'cleaning' dataframe is: ")
    display(df_customers_id_corretti)
    


# MERGE TRA TUTTI I DATAFRAME
- while merging dataframes, I drop non-standardized or uncleaned columns to keep only the cleaned ones.
- after merging, I rename the cleaned columns from '*_mod' to '*_def' to mark them as final versions.

In [ ]:
#MERGE TRA ORDERS E COUNTRIES SULLE CITTA'
if verbose >= 1:
    print("Merging the orders and countries dataframes using ['City_def', 'Postal Code_mod'] as keys.")


df_date_pulite2['Postal Code_mod'] = df_date_pulite2['Postal Code'].astype(str)
df_countries_fuzz_city_95['Postal Code_mod'] = df_countries_fuzz_city_95['Postal Code'].astype(str)

df_merge = pd.merge(df_date_pulite2.drop(columns=['Order Date','Ship Date','Ship Mode','Postal Code','Sales','Quantity','Discount','Profit','City_fuzz_90','City_fuzz_85','City','Product ID']), df_countries_fuzz_city_95.drop(columns=['City','Postal Code','State_fuzz_90','State_fuzz_80','State','City_fuzz_90']), how='left', on=(['City_def','Postal Code_mod']))
pd.set_option('display.max_columns', None) #visualizzo tutte le colonne dei df


df_merge=df_merge[['Order ID','Product ID_def','Customer ID','Country','Region','State_lev_1','City_def','Postal Code_mod','Order Date_def','Ship Date_def','Ship Mode_mod','Sales_mod','Profit_mod','Discount_mod','Quantity_mod']]
df_merge=df_merge.rename(columns={'State_lev_1':'State_def',
                                  'Customer ID':'Customer ID_def',
                                  'Postal Code_mod':'Postal Code_def',
                                  'Profit_mod':'Profit_def',
                                  'Sales_mod':'Sales_def',
                                  'Ship Mode_mod':'Ship Mode_def',
                                  'Discount_mod':'Discount_def',
                                  'Quantity_mod':'Quantity_def',
                                  })

if verbose==2:
    print("The merged dataframe is")
    display(df_merge.sample(5))

In [ ]:
#MERGE TRA IL DF CON GIA ORDERS E COUNTRIES E IL DF CUSTOMERS CON CHIAVE CUSTOMER ID_DEF
if verbose>=1:
    print("Merging customers dataframe with the orders-countries one, using 'Customer ID_def' as key")

df_merge_customers=df_merge.merge(df_customers_lev_2.drop(columns=['Segment','Customer ID','Customer Name']), on='Customer ID_def', how='left')
df_merge_customers=df_merge_customers.rename(columns={'Segment_mod':'Segment_def',
                                                      'Customer Name_mod':'Customer Name_def'})

if verbose==2:
    print('The merged dataframe is:')
    display(df_merge_customers.sample(5))


In [ ]:
#MERGE TRA DF GIA CON I 3 FOGLI E IL DF PRODUCTS CON CHIAVE PRODUCT ID_DEF
if verbose>=1:
    print("Merging orders-countries-customers dataframe with products one, using 'Porduct ID_def' as key")
    print("Cleaning malformed 'Product ID' values using reconstruction logic for missing category data.")
    print("if the category starts with 'OF' but is not 'OFF', then i add a 'F' and then go back to standardized format ")
    print("Then, I replace manually some others incorrect Product IDs (hardcoded corrections).")


df_products_sub_lev2=df_products_sub_lev2.rename(columns={'Product ID_mod':'Product ID_def'})

df_merge_def=df_merge_customers.merge(df_products_sub_lev2.drop(columns=['Product Name','Category','Sub-Category','Product ID']).drop_duplicates(), on='Product ID_def',how='left')


'''PULISCO ERRORI SU PRODUCT ID'''
df_merge_def=df_merge_def.rename(columns={'Product ID_def':'Product ID_mod'})
df_merge_def['Product ID']=df_merge_def['Product ID_mod'].copy()


for index, row in df_merge_def.iterrows():
    if pd.isna(row['Category_mod']):
        product_parts=row['Product ID_mod'].split('-')
        categoria=product_parts[0]
        sotto_categoria=product_parts[1]
        codice_numerico=product_parts[2]
        if categoria.startswith('OF') and categoria != 'OFF' and len(categoria) == 3:
            nuova_categoria=categoria[0]+'F'+categoria[1]
            ultima_lettera_categoria=categoria[2]
            
            nuova_sotto_categoria=ultima_lettera_categoria + sotto_categoria[:-1]
            ultima_lettera_sotto_categoria= sotto_categoria[-1]

            nuovo_codice_numerico=ultima_lettera_sotto_categoria + codice_numerico

            nuovo_product_id = f"{nuova_categoria}-{nuova_sotto_categoria}-{nuovo_codice_numerico}"

            # Applica la modifica
            df_merge_def.at[index, 'Product ID_mod'] = nuovo_product_id

#FACCIO IL REPLACE PER ERRORI CHE NON SO COME CORREGGERE
df_merge_def['Product ID_mod']=df_merge_def['Product ID_mod'].replace('FUR-BO-10000468','FUR-BO-1000468').replace('FUR-U1-0001935','FUR-FU-10001935').replace('OFF-BI-1000948','OFF-BI-10000948').replace('OFF-ST-10001128','OFF-ST-10001228').replace('OFF-21-0002049','OFF-BI-10002049').replace('FUR-BO-10002206','FUR-BO-1002206').replace('TEC-PH-O10002555','TEC-PH-10002555').replace('OFF-PA-1001274','OFF-PA-10001274').replace('OFF-PA-1003724','OFF-PA-10003724')

df_merge_def=df_merge_def.rename(columns={'Product ID_mod':'Product ID_def'})
df_merge_def=df_merge_def.drop(columns=['Product ID','Category_mod','Sub-Category_mod']).merge(df_products_sub_lev2.drop(columns=['Product Name','Category','Sub-Category','Product ID']).drop_duplicates(), on='Product ID_def',how='left')
df_merge_def=df_merge_def.rename(columns={'Category_mod':'Category_def', 'Sub-Category_mod':'Sub-Category_def'})

if verbose==2:
    print('The merged dataframe is:')
    display(df_merge_def.sample(5))



In [ ]:
'''AGGIUNGO COLONNA MESE, QUARTER, ANNO, DISTANZA IN GIORNI TRA SHIP DATE E ORDER DATE, TIPO DI GIORNO DELL'ORDINE(GIORNO DELLA SETTIMANA O DEL WEEKEND)'''
df_merge_def['Order Month'] = df_merge_def['Order Date_def'].dt.month_name()
df_merge_def['Order Year'] = df_merge_def['Order Date_def'].dt.year
df_merge_def['Order Quarter']=df_merge_def['Order Date_def'].dt.quarter
df_merge_def['Days between order date and shipping date']= df_merge_def['Ship Date_def'] - df_merge_def['Order Date_def']
df_merge_def['Order Day Type'] = df_merge_def['Order Date_def'].dt.dayofweek.apply(
    lambda x: 'Weekend' if x >= 5 else 'Weekday')
df_merge_def['Order Week']=df_merge_def['Order Date_def'].dt.isocalendar().week
df_merge_def['Order Day of the Week']=df_merge_def['Order Date_def'].dt.day_name()
#display(df_merge_def['Region'].unique())
#display(df_merge_def.loc[df_merge_def['Region'].isna()])

'''RENDO LE COLONNE TESTUALI DA TIPO OBJECT A TIPO STR'''
df_merge_def['Days between order date and shipping date']=df_merge_def['Days between order date and shipping date'].dt.days
display(df_merge_def['Days between order date and shipping date'].unique())
display(df_merge_def.columns)






array([ 5.,  4.,  3.,  2.,  0.,  7., nan,  1.,  6.])

CREO CSV CON IL DF FINALE

In [ ]:
df_merge_def.to_csv('../data/processed/MasterTable.csv', index=False)